In [1]:
import os
import re
import torch
from torch import nn
from torch.autograd import variable
import torchtext
from torchtext import data,datasets
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import spacy
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from functools import reduce
from sklearn.svm import SVC

In [32]:
file = '../data1_train_tree/apwsE950331.0603-0-1'
f = open(file)
article = []
for sentence in f:
    new_sentence = process(sentence)
    article.append(new_sentence)
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(article)
vectorizer.vocabulary_
X.toarray()
show = pd.DataFrame(data=X.toarray(),columns=vectorizer.vocabulary_)
show.index = ['sentence'+str(i) for i in range(1,len(show.index)+1)]
def transition_to_vector(t_n):
    columns = [(0,0),(0,1),(1,0),(1,1)]
    article_vector = np.array([])
    transitions = list()
    for c in range(t_n.shape[1]):
        for r in range(t_n.shape[0]-1):
            transitions.append((t_n[r,c],t_n[r+1,c]))
    article_counter = Counter(transitions)
    for index in columns:
        count = article_counter.get(index,0)
        article_vector = np.append(article_vector,np.array([count]))
    return (article_vector/article_vector.sum())[np.newaxis,:]
transition_to_vector(X.toarray())
pd.DataFrame(data=transition_to_vector(X.toarray()),columns=['0->0','0->1','1->0','1->1'])

,0->0,0->1,1->0,1->1
0,0.627551,0.147959,0.168367,0.056122


In [48]:
pd.read_csv("../finaldataset1/earthquake_train.csv",header=None).shape

(2980, 2)

# Baseline model ---- Entity Grid 
<a href="https://people.csail.mit.edu/regina/my_papers/coherence.pdf">Modeling Local Coherence: An Entity-Based Approach</a>

The data I collected had already been parsed.
<br>

For example: 

(ROOT(S(S(NP (DT No) (NN damage))(VP (VBD was)(VP(ADVP (RB immediately))(VBN reported)(PP (IN in)(NP (NNP Manila)))(, ,)(SBAR(WHADVP (WRB where))(S(NP (JJ tall) (NNS buildings))(VP (VBD were)(VP (VBN jolted)(CC and)(VBN shaken)(PP (IN by)(NP (DT the) (NN temblor))))))))))(, ,)(CC or)(S(NP(NP (JJ other) (NNS areas))(PP (IN outside)(NP(NP (NNP Mindoro) (NNP Island))(, ,)(NP(NP (DT a) (NN police) (NN spokesman))(PP (IN in)(NP (NNP Manila)))))))(VP (VBD said)))(. .)))

In [3]:
def process(text):
    '''This function will extract all of the noun in a sentence, 
    then put them together into a string seperated by white splace'''
    result = re.findall('\((NNP|NNPS|NNS|NN|PRP|NN) ([A-Za-z-]+)',text)
    return ' '.join(map(lambda x : x[1].lower(),result))
test_str = '''(ROOT(S(S(NP (DT No) (NN damage))(VP (VBD was)(VP(ADVP (RB immediately))(VBN reported)(PP (IN in)(NP (NNP Manila)))(, ,)(SBAR(WHADVP (WRB where))(S(NP (JJ tall) (NNS buildings))(VP (VBD were)(VP (VBN jolted)(CC and)(VBN shaken)(PP (IN by)(NP (DT the) (NN temblor))))))))))(, ,)(CC or)(S(NP(NP (JJ other) (NNS areas))(PP (IN outside)(NP(NP (NNP Mindoro) (NNP Island))(, ,)(NP(NP (DT a) (NN police) (NN spokesman))(PP (IN in)(NP (NNP Manila)))))))(VP (VBD said)))(. .)))'''
process(test_str)

'damage manila buildings temblor areas mindoro island police spokesman manila'

The following code will transfer all of the data sets from parsed sentences to entity grid by CountVectorizer

In [16]:

paths = ["../data1_test_tree/","../data1_train_tree/",
       "../data2_test_tree/","../data2_train_tree/"]
to_paths = path = ["../data1_test_numpy/","../data1_train_numpy/",
       "../data2_test_numpy/","../data2_train_numpy/"]
for path,to_path in zip(paths,to_paths):
    for file in list(filter(lambda file: re.findall('^[a-zA-Z0-9]',file),os.listdir(path))):
        if path in ["../data1_test_tree/","../data2_test_tree/"]:
            if file.endswith("perm-0"):
                f = open(path+file)
                article = []
                for sentence in f:
                    new_sentence = process(sentence)
                    article.append(new_sentence)
                vectorizer = CountVectorizer(binary=True)
                X = vectorizer.fit_transform(article)
                pickle.dump(X.toarray(), open(to_path+file, "wb" ))
        else : 
            f = open(path+file)
            article = []
            for sentence in f:
                new_sentence = process(sentence)
                article.append(new_sentence)
            vectorizer = CountVectorizer(binary=True)
            X = vectorizer.fit_transform(article)
            pickle.dump(X.toarray(), open(to_path+file, "wb" ))

In [6]:
example = pickle.load(open('../data1_test_numpy/apwsE950407.0558-2-2-0.perm-0','rb'))
example

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
      

Now each article has been transfered to an entity grid, then the following function will convert the entity grid to a vector which shows how all of the nouns are transfer between sentences within the article

In [27]:
def transition_to_vector(t_n):
    columns = [(0,0),(0,1),(1,0),(1,1)]
    article_vector = np.array([])
    transitions = list()
    for c in range(t_n.shape[1]):
        for r in range(t_n.shape[0]-1):
            transitions.append((t_n[r,c],t_n[r+1,c]))
    article_counter = Counter(transitions)
    for index in columns:
        count = article_counter.get(index,0)
        article_vector = np.append(article_vector,np.array([count]))
    return (article_vector/article_vector.sum())[np.newaxis,:]

In [8]:
transition_to_vector(example)

array([[0.73375262, 0.11949686, 0.12997904, 0.01677149]])

The following code will generate maximum 20 permutation of the sentences from each of the original articles.

In [17]:
paths = ['../data1_test_numpy/','../data1_train_numpy/',
        '../data2_test_numpy/','../data2_train_numpy/']
save_names = ['data1_test.txt','data1_train.txt','data2_test.txt','data2_train.txt']
for path,save_name in zip(paths,save_names):
    #path = "./data1_test_tree_numpy/"
    data = None
    for file in list(filter(lambda file: re.findall('^[a-zA-Z0-9]',file),os.listdir(path))):
        f = open(path+file,'rb')
        article_transition = pickle.load(f)
        sentence_len = article_transition.shape[0]
        origin_article_vector = transition_to_vector(article_transition)
        origin_article_vector = np.append(origin_article_vector,np.array([[1]]),axis=1)
        index = np.arange(sentence_len)
        origin_index = np.arange(sentence_len)
        origin_index = origin_index[np.newaxis,:]
        if sentence_len < 8:
            permutation = 10
        else:
            permutation = 20
        i = 0
        while i < permutation:
            np.random.shuffle(index)
            index = index[::-1]
            np.random.shuffle(index)
            if (((origin_index==index).sum(axis=1))== sentence_len).sum() == 1:
                print('same order,try again')
            else:
                i = i+1
                origin_index = np.vstack([origin_index,index])
                permutation_article_transition = article_transition[index,:]
                permutation_vector = transition_to_vector(permutation_article_transition)
                permutation_vector = np.append(permutation_vector,np.array([[0]]),axis=1)
                if data is None:
                    data = np.copy(origin_article_vector)
                else:
                    data = np.append(data,origin_article_vector,axis=0)
                data = np.append(data,permutation_vector,axis=0)
    #print(data.shape)
    pickle.dump(data, open('../finalnumpy/'+save_name, "wb" ))


same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again
same order,try again


In [14]:
earthquake_test = open('../finalnumpy/data1_test.txt','rb')
earthquake_test = pickle.load(earthquake_test)

earthquake_train = open('../finalnumpy/data1_train.txt','rb')
earthquake_train = pickle.load(earthquake_train)

airplane_train = open('../finalnumpy/data2_train.txt','rb')
airplane_train = pickle.load(airplane_train)

airplane_test = open('../finalnumpy/data2_test.txt','rb')
airplane_test = pickle.load(airplane_test)

Train earthquake article

In [27]:
parameters = {
    'C' : [10**i for i in range(-5,2)],
    'kernel':[ 'linear', 'poly', 'rbf'],
    'tol':[1e-3,1e-4]
}
svc = SVC(probability=True)
clf = GridSearchCV(svc, parameters,cv=5)
clf.fit(airplane_train[:,:-1],airplane_train[:,-1])

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf'], 'tol': [0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Test earthquake article

In [29]:
pred = clf.best_estimator_.predict_proba(airplane_test[:,:-1])[:,1]
a = pred[::2]
b = pred[1::2]
correct_rate = ((a>b).sum())/len(a)
correct_rate

0.8386934673366834

Train airplane articles

In [15]:
parameters = {
    'C' : [10**i for i in range(-5,2)],
    'kernel':[ 'linear', 'poly', 'rbf'],
    'tol':[1e-3,1e-4]
}
svc = SVC(probability=True)
clf = GridSearchCV(svc, parameters,cv=5,n_jobs=-1)
clf.fit(earthquake_train[:,:-1],earthquake_train[:,-1])

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf'], 'tol': [0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Test airplace articles

In [16]:
pred = clf.best_estimator_.predict_proba(earthquake_test[:,:-1])[:,1]
a = pred[::2]
b = pred[1::2]
correct_rate = ((a>b).sum())/len(a)
correct_rate

0.8034682080924855

<h1> TreeRnn + GRN </h1>

# convert parsed sentences to orignal sentence

In [15]:
line = "(ROOT(S(ADVP(NP(QP (RB About) (CD 3))(NNS minutes))(RB later))(, ,)(NP (PRP he))(VP (VBD was)(VP (VBN told)(SBAR(S(NP (PRP$ his) (NN destination) (NN airport))(VP (VBD was)(PP (IN at)(NP(NP (CD 12) (RB o'clock))(CC and)(NP(NP (CD 8) (NNS miles))(, ,)(SBAR(WHNP (WDT which))(S(NP (PRP he))(VP (VBD acknowledged))))))))))))(. .)))"
text = re.sub(r'\s*(\([A-Z,.!#|$:;\'``"-]+)|(\))\s*', '', line)
text[1:]

"About 3 minutes later , he was told his destination airport was at 12 o'clock and 8 miles , which he acknowledged ."

In [16]:
path = "../data2_train_tree/"
new_path = "../data2_train_tree_revised/"
for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(path))):
    f = open(path+name)
    w = open(new_path+name,'w')
    for text in f:
        text = re.sub(r'\s*(\([A-Z,.!#|$:;\'``"-]+)|(\))\s*', '', text)
        text = text[1:]
        w.write(text.strip()+"\n")
    w.close()
    
path = "../data1_train_tree/"
new_path = "../data1_train_tree_revised/"
for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(path))):
    f = open(path+name)
    w = open(new_path+name,'w')
    for text in f:
        text = re.sub(r'\s*(\([A-Z,.!#|$:;\'``"-]+)|(\))\s*', '', text)
        text = text[1:]
        w.write(text.strip()+"\n")
    w.close()

In [17]:
path = "../data2_test_tree/"
new_path = "../data2_test_tree_revised/"
for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(path))):
    if re.findall(".*.perm-0$",name):
        f = open(path+name)
        w = open(new_path+name,'w')
        for text in f:
            text = re.sub(r'\s*(\([A-Z,.!#|$:;\'``"-]+)|(\))\s*', '', text)
            text = text[1:]
            w.write(text.strip()+"\n")
        w.close()
    
path = "../data1_test_tree/"
new_path = "../data1_test_tree_revised/"
for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(path))):
    if re.findall(".*.perm-0$",name):
        f = open(path+name)
        w = open(new_path+name,'w')
        for text in f:
            text = re.sub(r'\s*(\([A-Z,.!#|$:;\'``"-]+)|(\))\s*', '', text)
            text = text[1:]
            w.write(text.strip()+"\n")
        w.close()

# Call edu.stanford.nlp.sentiment.SentimentPipeline to generate pennTrees

In [3]:
dirpaths = ['../data2_train_tree_revised/','../data2_test_tree_revised/',
         '../data1_train_tree_revised/','../data1_test_tree_revised/']
to_dirpaths = ['../data2_train_tree_binary1/','../data2_test_tree_binary1/',
         '../data1_train_tree_binary1/','../data1_test_tree_binary1/']

for dirpath,to_dirpath in zip(dirpaths,to_dirpaths):
    filenames = (list(filter(lambda name: re.findall('^[a-z0-9A-Z].*',name),os.listdir(dirpath))))
    for filename in filenames:
        os.system('java -cp "*" edu.stanford.nlp.sentiment.SentimentPipeline -file {} -output pennTrees > {}'.format(dirpath+filename,to_dirpath+filename))


<h3> example of the result </h3>
BC-Philippines-Quake , 2nd Ld-Writethru | Strong Quake Rocks Philippines Island , 25 Deaths Reported | EDS : New thruout to UPDATE with radio report saying 25 people died , | many buildings collapsed , island still recovering from a typhoon | that struck last month .
(0 (1 (1 (2 (2 BC-Philippines-Quake) (2 (2 ,) (2 (2 2nd) (2 Ld-Writethru)))) (1 (1 (2 |) (1 (1 (1 (2 (2 (2 (2 Strong) (2 Quake)) (2 (2 Rocks) (2 (2 Philippines) (2 Island)))) (2 ,)) (2 (2 25) (2 (2 Deaths) (2 (2 Reported) (2 (2 |) (2 EDS)))))) (2 :)) (2 (3 New) (2 thruout)))) (2 (2 to) (2 (2 UPDATE) (2 (2 with) (2 (2 (2 radio) (2 report)) (2 (2 saying) (2 (2 25) (2 people))))))))) (1 died)) (1 (2 ,) (1 (2 (2 |) (2 (2 many) (2 buildings))) (1 (1 (2 (2 collapsed) (2 ,)) (1 (2 island) (1 (2 still) (1 (2 recovering) (2 (2 from) (2 (2 (2 a) (2 (2 typhoon) (2 |))) (2 (2 that) (2 (2 struck) (2 (2 last) (3 month)))))))))) (2 .)))))
No pickup .
(2 (1 No) (2 (2 pickup) (2 .)))
| By THOMAS WAGNER | Associated Press Writer .
(1 (1 (2 |) (1 (2 By) (1 (2 THOMAS) (1 (2 WAGNER) (1 (2 |) (1 (2 Associated) (2 (2 Press) (2 Writer)))))))) (2 .))
MANILA , Philippines -LRB- AP -RRB- A strong earthquake rocked the Philippine island of Mindoro early Tuesday , reportedly killing at least 25 people and destroying many homes .
(1 (3 (2 (2 MANILA) (2 ,)) (3 (2 (2 Philippines) (2 (2 -LRB-) (2 (2 AP) (2 -RRB-)))) (3 (2 A) (3 (3 strong) (2 earthquake))))) (1 (1 (1 (1 (2 (2 rocked) (2 (2 (2 the) (2 (2 Philippine) (2 island))) (2 (2 of) (2 Mindoro)))) (2 (2 early) (2 Tuesday))) (2 ,)) (1 (2 reportedly) (1 (1 (1 (2 killing) (2 (2 (2 (2 at) (1 least)) (2 25)) (2 people))) (2 and)) (2 (2 destroying) (2 (2 many) (2 homes)))))) (2 .)))
DZRH Radio , with a reporter on the scene , said most of those people died while sleeping in buildings that collapsed , including a structure that houses the Philippine Coast Guard .
(1 (1 (1 (2 (2 (2 DZRH) (2 Radio)) (2 ,)) (2 (2 with) (2 (2 (2 a) (2 reporter)) (2 (2 on) (2 (2 the) (2 scene)))))) (2 ,)) (2 (2 (2 said) (1 (2 (2 most) (2 (2 of) (2 (2 those) (2 people)))) (1 (2 (2 (1 died) (2 (2 while) (2 (2 sleeping) (2 (2 in) (2 (2 buildings) (2 (2 that) (2 collapsed))))))) (2 ,)) (2 (2 including) (3 (2 (2 a) (2 structure)) (2 (2 that) (2 (2 houses) (2 (2 the) (2 (2 Philippine) (2 (2 Coast) (2 Guard))))))))))) (2 .)))
Mindoro Island is still recovering from a typhoon that struck the area last month , destroying much of the coconut and rice that famrs grow in small fields there .
(1 (2 (2 Mindoro) (2 Island)) (1 (1 (2 (2 is) (2 still)) (1 (2 recovering) (1 (2 from) (1 (2 (2 a) (2 typhoon)) (1 (2 that) (1 (2 (2 (2 (2 struck) (2 (2 the) (2 area))) (2 (2 last) (3 month))) (2 ,)) (2 (2 (2 destroying) (2 (2 much) (2 (2 of) (2 (2 the) (2 (2 (2 coconut) (2 and)) (2 rice)))))) (2 (2 that) (2 (2 famrs) (2 (2 (2 grow) (2 (2 in) (2 (2 small) (2 fields)))) (2 there))))))))))) (2 .)))
The quake , which occurred at 3:15 am -LRB- 14:15 Monday GMT -RRB- , had a preliminary magnitude of 6.7 and was centered in a strait 10 kilometers -LRB- six miles -RRB- north of Calapan , a town on the northern end of the island , said the Philippine Institute of Vulcanology and Seismology .
(1 (1 (1 (2 (2 (2 The) (2 quake)) (2 ,)) (1 (2 which) (1 (1 (1 (1 (2 (2 occurred) (2 (2 at) (1 (2 (2 3:15) (2 am)) (2 (2 -LRB-) (2 (2 (2 14:15) (2 (2 Monday) (2 GMT))) (2 -RRB-)))))) (2 ,)) (2 (2 had) (2 (3 (2 a) (2 (2 preliminary) (2 magnitude))) (2 (2 of) (2 6.7))))) (2 and)) (2 (2 was) (2 (2 centered) (1 (2 in) (1 (1 (2 a) (1 (2 (2 strait) (2 (3 10) (2 kilometers))) (2 (2 (2 -LRB-) (2 (2 (2 six) (2 miles)) (2 -RRB-))) (2 north)))) (2 (2 of) (3 (2 (2 Calapan) (2 ,)) (2 (2 (2 a) (3 town)) (2 (2 on) (2 (2 (2 the) (2 (2 northern) (2 end))) (2 (2 of) (2 (2 the) (2 island))))))))))))))) (2 ,)) (2 (2 (2 said) (2 (2 (2 the) (2 (2 Philippine) (2 Institute))) (2 (2 of) (2 (2 (2 Vulcanology) (2 and)) (2 Seismology))))) (2 .)))
Lasting about four minutes , the quake was felt at intensities of 3 and 4 throughout central and south Philippines , including Manila , 75 miles -LRB- 120 kilometers -RRB- north of the epicenter , said Ted Sandoval , a specialist at the institute .
(0 (1 (1 (1 (2 (2 Lasting) (2 (2 (2 about) (2 four)) (2 minutes))) (1 (2 ,) (1 (1 (2 (2 the) (2 quake)) (1 (2 was) (1 (1 (2 (2 (2 felt) (2 (2 at) (2 (2 intensities) (2 (2 of) (2 (2 (2 3) (2 and)) (2 4)))))) (2 (2 throughout) (2 (2 central) (2 (2 and) (2 (2 south) (2 Philippines)))))) (2 ,)) (1 (2 including) (1 (2 (2 Manila) (2 ,)) (1 (1 (2 (2 75) (2 miles)) (2 (2 -LRB-) (2 (2 (2 120) (2 kilometers)) (2 -RRB-)))) (2 (2 north) (2 (2 of) (2 (2 the) (2 epicenter)))))))))) (2 ,)))) (2 said)) (1 (1 (2 (2 Ted) (2 Sandoval)) (2 ,)) (2 (2 (2 a) (2 specialist)) (2 (2 at) (2 (2 the) (2 institute)))))) (2 .))
No damage was immediately reported in Manila , where tall buildings were jolted and shaken by the temblor , or other areas outside Mindoro Island , a police spokesman in Manila said .
(1 (1 (1 (1 (1 (2 (1 No) (1 damage)) (1 (2 was) (1 (2 immediately) (1 (2 (2 (2 reported) (2 (2 in) (2 Manila))) (2 ,)) (2 (2 where) (2 (2 (2 tall) (2 buildings)) (2 (2 were) (2 (2 (2 (2 jolted) (2 and)) (2 shaken)) (2 (2 by) (2 (2 the) (2 temblor))))))))))) (2 ,)) (2 or)) (2 (2 (2 (2 other) (2 areas)) (2 (2 outside) (2 (2 (2 (2 Mindoro) (2 Island)) (2 ,)) (2 (2 (2 a) (2 (2 police) (2 spokesman))) (2 (2 in) (2 Manila)))))) (2 said))) (2 .))
The U.S. Geological Survey in Menlo Park , Calif. , put the quake 's preliminary magnitude at 7.1 .
(1 (1 (1 (2 (2 The) (2 (2 U.S.) (2 (2 Geological) (2 Survey)))) (1 (2 in) (1 (1 (2 (2 (2 (2 Menlo) (2 Park)) (2 ,)) (2 Calif.)) (2 ,)) (2 put)))) (2 (2 (2 (2 the) (2 (2 quake) (2 's))) (2 (2 preliminary) (2 magnitude))) (2 (2 at) (2 7.1)))) (2 .))
Before phone lines were cut off on Mindoro Island , Gov. Rodolfo Valencia of Oriental Mindoro province told DZBB and DZMM radio that at least six people died when their homes collapsed or were washed away by large waves , and three bridges were damaged and closed .
(1 (2 (2 Before) (2 (2 (2 phone) (2 lines)) (2 (2 were) (2 (2 (2 cut) (1 off)) (2 (2 on) (2 (2 Mindoro) (2 Island))))))) (1 (2 ,) (1 (1 (1 (1 (1 (1 (2 (2 Gov.) (2 (2 Rodolfo) (2 Valencia))) (2 (2 of) (2 (2 Oriental) (2 (2 Mindoro) (2 province))))) (2 (2 told) (2 (2 DZBB) (2 (2 and) (2 (2 DZMM) (2 (2 radio) (1 (2 that) (1 (2 (2 (2 (2 at) (1 least)) (2 six)) (2 people)) (2 (1 died) (2 (2 when) (2 (2 (2 their) (2 homes)) (1 (2 (2 collapsed) (2 or)) (1 (2 were) (3 (3 (2 washed) (2 away)) (2 (2 by) (2 (2 large) (2 waves))))))))))))))))) (2 ,)) (2 and)) (2 (2 (2 three) (2 bridges)) (2 (2 were) (2 (2 (1 damaged) (2 and)) (2 closed))))) (2 .))))
The DZRH reporter called Manila on a cellular phone .
(1 (2 (2 The) (2 (2 DZRH) (2 reporter))) (2 (2 (2 called) (2 (2 Manila) (2 (2 on) (2 (2 a) (2 (2 cellular) (2 phone)))))) (2 .)))
But Sandoval said no tidal waves were reported or expected from the quake , one of the strongest ever to occur on the Lubang fault , one of the area 's most active .
(1 (1 (2 But) (2 (2 Sandoval) (1 (2 said) (1 (2 (1 no) (2 (3 tidal) (2 waves))) (2 (2 were) (2 (3 (2 (2 (2 reported) (2 or)) (1 expected)) (3 (2 from) (3 (2 (2 (2 the) (2 quake)) (2 ,)) (3 (2 one) (3 (2 of) (3 (3 (2 the) (3 strongest)) (2 ever))))))) (2 (2 to) (2 (2 occur) (2 (2 on) (2 (2 the) (1 (2 Lubang) (2 fault)))))))))))) (2 (2 ,) (3 (2 (2 one) (2 (2 of) (2 (2 the) (2 area)))) (2 (2 (2 's) (2 (2 most) (2 active))) (2 .)))))
A magnitude 6 quake can cause severe damage if centered under a populated area , while amgnitude 7 quake indicates a major quake capable of widespread , heavy damage .
(1 (2 (2 A) (2 (2 magnitude) (2 (2 6) (2 quake)))) (1 (1 (2 can) (1 (1 (2 cause) (2 (2 severe) (1 damage))) (1 (2 if) (1 (2 (2 (2 centered) (2 (2 under) (2 (2 a) (3 (2 populated) (2 area))))) (2 ,)) (1 (2 while) (1 (2 (2 amgnitude) (2 (2 7) (2 quake))) (2 (2 indicates) (1 (2 (2 a) (2 (2 major) (2 quake))) (1 (3 capable) (2 (2 of) (2 (2 widespread) (2 (2 ,) (2 (2 heavy) (1 damage)))))))))))))) (2 .)))


# Convert the format of parsed sentences to the format that is more suitable to bulid recursive nn

In [18]:
text = '''BC-Philippines-Quake , 2nd Ld-Writethru | Strong Quake Rocks Philippines Island , 25 Deaths Reported | EDS : New thruout to UPDATE with radio report saying 25 people died , | many buildings collapsed , island still recovering from a typhoon | that struck last month .
(0 (1 (1 (2 (2 BC-Philippines-Quake) (2 (2 ,) (2 (2 2nd) (2 Ld-Writethru)))) (1 (1 (2 |) (1 (1 (1 (2 (2 (2 (2 Strong) (2 Quake)) (2 (2 Rocks) (2 (2 Philippines) (2 Island)))) (2 ,)) (2 (2 25) (2 (2 Deaths) (2 (2 Reported) (2 (2 |) (2 EDS)))))) (2 :)) (2 (3 New) (2 thruout)))) (2 (2 to) (2 (2 UPDATE) (2 (2 with) (2 (2 (2 radio) (2 report)) (2 (2 saying) (2 (2 25) (2 people))))))))) (1 died)) (1 (2 ,) (1 (2 (2 |) (2 (2 many) (2 buildings))) (1 (1 (2 (2 collapsed) (2 ,)) (1 (2 island) (1 (2 still) (1 (2 recovering) (2 (2 from) (2 (2 (2 a) (2 (2 typhoon) (2 |))) (2 (2 that) (2 (2 struck) (2 (2 last) (3 month)))))))))) (2 .)))))
No pickup .
(2 (1 No) (2 (2 pickup) (2 .)))
| By THOMAS WAGNER | Associated Press Writer .
(1 (1 (2 |) (1 (2 By) (1 (2 THOMAS) (1 (2 WAGNER) (1 (2 |) (1 (2 Associated) (2 (2 Press) (2 Writer)))))))) (2 .))
MANILA , Philippines -LRB- AP -RRB- A strong earthquake rocked the Philippine island of Mindoro early Tuesday , reportedly killing at least 25 people and destroying many homes .
(1 (3 (2 (2 MANILA) (2 ,)) (3 (2 (2 Philippines) (2 (2 -LRB-) (2 (2 AP) (2 -RRB-)))) (3 (2 A) (3 (3 strong) (2 earthquake))))) (1 (1 (1 (1 (2 (2 rocked) (2 (2 (2 the) (2 (2 Philippine) (2 island))) (2 (2 of) (2 Mindoro)))) (2 (2 early) (2 Tuesday))) (2 ,)) (1 (2 reportedly) (1 (1 (1 (2 killing) (2 (2 (2 (2 at) (1 least)) (2 25)) (2 people))) (2 and)) (2 (2 destroying) (2 (2 many) (2 homes)))))) (2 .)))
DZRH Radio , with a reporter on the scene , said most of those people died while sleeping in buildings that collapsed , including a structure that houses the Philippine Coast Guard .
(1 (1 (1 (2 (2 (2 DZRH) (2 Radio)) (2 ,)) (2 (2 with) (2 (2 (2 a) (2 reporter)) (2 (2 on) (2 (2 the) (2 scene)))))) (2 ,)) (2 (2 (2 said) (1 (2 (2 most) (2 (2 of) (2 (2 those) (2 people)))) (1 (2 (2 (1 died) (2 (2 while) (2 (2 sleeping) (2 (2 in) (2 (2 buildings) (2 (2 that) (2 collapsed))))))) (2 ,)) (2 (2 including) (3 (2 (2 a) (2 structure)) (2 (2 that) (2 (2 houses) (2 (2 the) (2 (2 Philippine) (2 (2 Coast) (2 Guard))))))))))) (2 .)))
Mindoro Island is still recovering from a typhoon that struck the area last month , destroying much of the coconut and rice that famrs grow in small fields there .
(1 (2 (2 Mindoro) (2 Island)) (1 (1 (2 (2 is) (2 still)) (1 (2 recovering) (1 (2 from) (1 (2 (2 a) (2 typhoon)) (1 (2 that) (1 (2 (2 (2 (2 struck) (2 (2 the) (2 area))) (2 (2 last) (3 month))) (2 ,)) (2 (2 (2 destroying) (2 (2 much) (2 (2 of) (2 (2 the) (2 (2 (2 coconut) (2 and)) (2 rice)))))) (2 (2 that) (2 (2 famrs) (2 (2 (2 grow) (2 (2 in) (2 (2 small) (2 fields)))) (2 there))))))))))) (2 .)))
The quake , which occurred at 3:15 am -LRB- 14:15 Monday GMT -RRB- , had a preliminary magnitude of 6.7 and was centered in a strait 10 kilometers -LRB- six miles -RRB- north of Calapan , a town on the northern end of the island , said the Philippine Institute of Vulcanology and Seismology .
(1 (1 (1 (2 (2 (2 The) (2 quake)) (2 ,)) (1 (2 which) (1 (1 (1 (1 (2 (2 occurred) (2 (2 at) (1 (2 (2 3:15) (2 am)) (2 (2 -LRB-) (2 (2 (2 14:15) (2 (2 Monday) (2 GMT))) (2 -RRB-)))))) (2 ,)) (2 (2 had) (2 (3 (2 a) (2 (2 preliminary) (2 magnitude))) (2 (2 of) (2 6.7))))) (2 and)) (2 (2 was) (2 (2 centered) (1 (2 in) (1 (1 (2 a) (1 (2 (2 strait) (2 (3 10) (2 kilometers))) (2 (2 (2 -LRB-) (2 (2 (2 six) (2 miles)) (2 -RRB-))) (2 north)))) (2 (2 of) (3 (2 (2 Calapan) (2 ,)) (2 (2 (2 a) (3 town)) (2 (2 on) (2 (2 (2 the) (2 (2 northern) (2 end))) (2 (2 of) (2 (2 the) (2 island))))))))))))))) (2 ,)) (2 (2 (2 said) (2 (2 (2 the) (2 (2 Philippine) (2 Institute))) (2 (2 of) (2 (2 (2 Vulcanology) (2 and)) (2 Seismology))))) (2 .)))
Lasting about four minutes , the quake was felt at intensities of 3 and 4 throughout central and south Philippines , including Manila , 75 miles -LRB- 120 kilometers -RRB- north of the epicenter , said Ted Sandoval , a specialist at the institute .
(0 (1 (1 (1 (2 (2 Lasting) (2 (2 (2 about) (2 four)) (2 minutes))) (1 (2 ,) (1 (1 (2 (2 the) (2 quake)) (1 (2 was) (1 (1 (2 (2 (2 felt) (2 (2 at) (2 (2 intensities) (2 (2 of) (2 (2 (2 3) (2 and)) (2 4)))))) (2 (2 throughout) (2 (2 central) (2 (2 and) (2 (2 south) (2 Philippines)))))) (2 ,)) (1 (2 including) (1 (2 (2 Manila) (2 ,)) (1 (1 (2 (2 75) (2 miles)) (2 (2 -LRB-) (2 (2 (2 120) (2 kilometers)) (2 -RRB-)))) (2 (2 north) (2 (2 of) (2 (2 the) (2 epicenter)))))))))) (2 ,)))) (2 said)) (1 (1 (2 (2 Ted) (2 Sandoval)) (2 ,)) (2 (2 (2 a) (2 specialist)) (2 (2 at) (2 (2 the) (2 institute)))))) (2 .))
No damage was immediately reported in Manila , where tall buildings were jolted and shaken by the temblor , or other areas outside Mindoro Island , a police spokesman in Manila said .
(1 (1 (1 (1 (1 (2 (1 No) (1 damage)) (1 (2 was) (1 (2 immediately) (1 (2 (2 (2 reported) (2 (2 in) (2 Manila))) (2 ,)) (2 (2 where) (2 (2 (2 tall) (2 buildings)) (2 (2 were) (2 (2 (2 (2 jolted) (2 and)) (2 shaken)) (2 (2 by) (2 (2 the) (2 temblor))))))))))) (2 ,)) (2 or)) (2 (2 (2 (2 other) (2 areas)) (2 (2 outside) (2 (2 (2 (2 Mindoro) (2 Island)) (2 ,)) (2 (2 (2 a) (2 (2 police) (2 spokesman))) (2 (2 in) (2 Manila)))))) (2 said))) (2 .))
The U.S. Geological Survey in Menlo Park , Calif. , put the quake 's preliminary magnitude at 7.1 .
(1 (1 (1 (2 (2 The) (2 (2 U.S.) (2 (2 Geological) (2 Survey)))) (1 (2 in) (1 (1 (2 (2 (2 (2 Menlo) (2 Park)) (2 ,)) (2 Calif.)) (2 ,)) (2 put)))) (2 (2 (2 (2 the) (2 (2 quake) (2 's))) (2 (2 preliminary) (2 magnitude))) (2 (2 at) (2 7.1)))) (2 .))
Before phone lines were cut off on Mindoro Island , Gov. Rodolfo Valencia of Oriental Mindoro province told DZBB and DZMM radio that at least six people died when their homes collapsed or were washed away by large waves , and three bridges were damaged and closed .
(1 (2 (2 Before) (2 (2 (2 phone) (2 lines)) (2 (2 were) (2 (2 (2 cut) (1 off)) (2 (2 on) (2 (2 Mindoro) (2 Island))))))) (1 (2 ,) (1 (1 (1 (1 (1 (1 (2 (2 Gov.) (2 (2 Rodolfo) (2 Valencia))) (2 (2 of) (2 (2 Oriental) (2 (2 Mindoro) (2 province))))) (2 (2 told) (2 (2 DZBB) (2 (2 and) (2 (2 DZMM) (2 (2 radio) (1 (2 that) (1 (2 (2 (2 (2 at) (1 least)) (2 six)) (2 people)) (2 (1 died) (2 (2 when) (2 (2 (2 their) (2 homes)) (1 (2 (2 collapsed) (2 or)) (1 (2 were) (3 (3 (2 washed) (2 away)) (2 (2 by) (2 (2 large) (2 waves))))))))))))))))) (2 ,)) (2 and)) (2 (2 (2 three) (2 bridges)) (2 (2 were) (2 (2 (1 damaged) (2 and)) (2 closed))))) (2 .))))
The DZRH reporter called Manila on a cellular phone .
(1 (2 (2 The) (2 (2 DZRH) (2 reporter))) (2 (2 (2 called) (2 (2 Manila) (2 (2 on) (2 (2 a) (2 (2 cellular) (2 phone)))))) (2 .)))
But Sandoval said no tidal waves were reported or expected from the quake , one of the strongest ever to occur on the Lubang fault , one of the area 's most active .
(1 (1 (2 But) (2 (2 Sandoval) (1 (2 said) (1 (2 (1 no) (2 (3 tidal) (2 waves))) (2 (2 were) (2 (3 (2 (2 (2 reported) (2 or)) (1 expected)) (3 (2 from) (3 (2 (2 (2 the) (2 quake)) (2 ,)) (3 (2 one) (3 (2 of) (3 (3 (2 the) (3 strongest)) (2 ever))))))) (2 (2 to) (2 (2 occur) (2 (2 on) (2 (2 the) (1 (2 Lubang) (2 fault)))))))))))) (2 (2 ,) (3 (2 (2 one) (2 (2 of) (2 (2 the) (2 area)))) (2 (2 (2 's) (2 (2 most) (2 active))) (2 .)))))
A magnitude 6 quake can cause severe damage if centered under a populated area , while amgnitude 7 quake indicates a major quake capable of widespread , heavy damage .
(1 (2 (2 A) (2 (2 magnitude) (2 (2 6) (2 quake)))) (1 (1 (2 can) (1 (1 (2 cause) (2 (2 severe) (1 damage))) (1 (2 if) (1 (2 (2 (2 centered) (2 (2 under) (2 (2 a) (3 (2 populated) (2 area))))) (2 ,)) (1 (2 while) (1 (2 (2 amgnitude) (2 (2 7) (2 quake))) (2 (2 indicates) (1 (2 (2 a) (2 (2 major) (2 quake))) (1 (3 capable) (2 (2 of) (2 (2 widespread) (2 (2 ,) (2 (2 heavy) (1 damage)))))))))))))) (2 .)))
'''

In [22]:
def text_refine(text):
    text = re.sub('\([0-9] ',  '(', text)
    def t(i):
        i = i.group(0)
        i = list(i)
        i.remove('(')
        i.remove(')')
        i = ''.join(i)
        return i
    go = True
    while go:
        original = len(text)
        text = re.sub(r'\s*\([^ \)\(\t\n\r\f\v]+\)\s*', t, text)
        go = not (original == len(text))
    return text
def extract_tree(text):
    binary_tree = ''
    count = 0
    i = text.find('(')
    while i < (len(text)):
        if text[i] == '(':
            if count == 0:
                start = i
            count = count +1
        elif text[i] == ')':
            count = count - 1
            if count == 0:
                end = i
                target = (text[start:end+1])
                if (len(target) > 5) and (list(target).count(')')>=2):
                     binary_tree = binary_tree + text_refine(target) + '\n'
        i = i+1
    return binary_tree.strip()

In [23]:
print(extract_tree(text))

((((BC-Philippines-Quake (, (2nd Ld-Writethru))) ((| ((((((Strong Quake) (Rocks (Philippines Island))) ,) (25 (Deaths (Reported (| EDS))))) :) (New thruout))) (to (UPDATE (with ((radio report) (saying (25 people)))))))) died) (, ((| (many buildings)) (((collapsed ,) (island (still (recovering (from ((a (typhoon |)) (that (struck (last month))))))))) .))))
(No (pickup .))
((| (By (THOMAS (WAGNER (| (Associated (Press Writer))))))) .)
(((MANILA ,) ((Philippines (-LRB- (AP -RRB-))) (A (strong earthquake)))) (((((rocked ((the (Philippine island)) (of Mindoro))) (early Tuesday)) ,) (reportedly (((killing (((at least) 25) people)) and) (destroying (many homes))))) .))
(((((DZRH Radio) ,) (with ((a reporter) (on (the scene))))) ,) ((said ((most (of (those people))) (((died (while (sleeping (in (buildings (that collapsed)))))) ,) (including ((a structure) (that (houses (the (Philippine (Coast Guard)))))))))) .))
((Mindoro Island) (((is still) (recovering (from ((a typhoon) (that ((((struck (th

In [24]:
to_dirpaths = ['../data2_train_tree_binary1/','../data2_test_tree_binary1/',
         '../data1_train_tree_binary1/','../data1_test_tree_binary1/']
new_paths = ['../data2_train_tree_binary/','../data2_test_tree_binary/',
         '../data1_train_tree_binary/','../data1_test_tree_binary/']

for path,new_path in zip(to_dirpaths,new_paths):
    for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(path))):
        f = open(path+name).read().strip()
        w = open(new_path+name,'w')
        w.write(extract_tree(f))
        w.close()

# Build vocabulary for both train and test dataset

In [125]:
test_and_train = pd.DataFrame(columns=['text'])
paths = ['./data1_test_tree_revised/','./data1_train_tree_revised/',
        './data2_test_tree_revised/','./data2_train_tree_revised/']
for path in paths:
    for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(path))):
        text = open(path+name)
        text = text.read()
        test_and_train = test_and_train.append({'text':text.strip()},ignore_index=True)
test_and_train.to_csv('all_text.csv',header=False,index=False)

In [5]:
def split_n(x):
    return x.split('\n')
#TEXT = data.Field(lower=True)
#NEST = data.NestedField(nesting_field=TEXT,tokenize=split_n)
#test_and_train = data.TabularDataset(path='./all_text.csv',format='csv',fields=[('nest', NEST)])

# Save index_to_voca, voca_to_index, and glove embedding in local file

In [70]:
NEST.build_vocab(test_and_train)
NEST.vocab.load_vectors('glove.42B.300d')
itos = open('./vocabulary/itos.pickle','wb')
stoi = open('./vocabulary/stoi.pickle','wb')
pickle.dump(NEST.vocab.itos,itos)
pickle.dump(NEST.vocab.stoi,stoi)
torch.save(NEST.vocab.vectors,'./vocabulary/embedding')

# Generate 20 random permutations for each article in bothing training and testing dataset, save the generated data into csv with the format of (text,binary tree, coherence), where 'text' is the plain text, 'binary tree' is corresponding binary parse tree, 'coherence' denote whether the order of the sentences is the original order or not.

In [163]:
earthquake_train =  pd.DataFrame(columns=['binary tree','coherence'])
earthquake_test = pd.DataFrame(columns=['binary tree','coherence'])
airplane_train =  pd.DataFrame(columns=['binary tree','coherence'])
airplane_test = pd.DataFrame(columns=['binary tree','coherence'])

In [164]:
def generate_20_permutation_and_save_csv(plain_text_paths,binary_tree_paths,dfs,savefiles):
    for plain_text_path,binary_tree_path,df,savefile in zip(plain_text_paths,binary_tree_paths,dfs,savefiles):
        for name in list(filter(lambda name: re.findall("^[a-zA-Z0-9]",name),os.listdir(binary_tree_path))):
            #plain_texts = open(plain_text_path+name)
            binary_trees = open(binary_tree_path+name)
            #orignial_texts = plain_texts = plain_texts.read().strip()
            orignial_trees = binary_trees = binary_trees.read().strip()
            #plain_texts = plain_texts.split('\n')
            binary_trees = binary_trees.split('\n')
            s_len = len(binary_trees)
            indices = np.arange(s_len)
            generated_indices = np.arange(s_len)[np.newaxis,:]
            time = 0
            if s_len < 8:
                permutation = 10
            else:
                permutation = 20
            while time<permutation:
                np.random.shuffle(indices)
                indices = indices[::-1]
                np.random.shuffle(indices)
                if (((generated_indices == indices).sum(axis=1)) == s_len).sum() == 1:
                    print('same order, try again')
                else:
                    df = df.append({'binary tree':orignial_trees.strip(),'coherence':1},ignore_index=True)
                    generated_indices = np.vstack([generated_indices,indices])
                    #new_text = ''
                    new_binary_tree = ''
                    for index in indices:
                        #new_text = new_text + plain_texts[index] + '\n'
                        new_binary_tree = new_binary_tree + binary_trees[index] + '\n'
                    df = df.append({'binary tree':new_binary_tree.strip(),'coherence':0},ignore_index=True)
                    time = time + 1
        df.to_csv(savefile,header=False,index=False)


In [166]:
plain_text_paths = ['../data1_test_tree_revised/','../data1_train_tree_revised/',
                   '../data2_test_tree_revised/','../data2_train_tree_revised/']
binary_tree_paths = ['../data1_test_tree_binary/','../data1_train_tree_binary/',
                    '../data2_test_tree_binary/','../data2_train_tree_binary/']
dfs = [earthquake_test,earthquake_train,airplane_test,airplane_train]
savefiles = ['../finaldataset1/earthquake_test.csv','../finaldataset1/earthquake_train.csv',
            '../finaldataset1/airplane_test.csv','../finaldataset1/airplane_train.csv']
generate_20_permutation_and_save_csv(plain_text_paths,binary_tree_paths,dfs,savefiles)

same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again
same order, try again


In [2]:
def fun(x):
    result = re.sub('\(','',x)
    result = re.sub('\)',' )',result)
    result = result.split()
    return list(map(lambda token: token.strip(),result))
def split_n(text):
    return text.split('\n')

In [16]:
class TreeRnnAndGRU(nn.Module):
    def __init__(self,p1=0.2):
        super().__init__()
        stoi = open('../vocabulary/stoi.pickle','rb')
        self.stoi = pickle.load(stoi)
        embedding = torch.load('../vocabulary/embedding')
        self.embedding = embedding.cuda()
        h_0 = torch.zeros(1,1,300)
        h_0 = torch.normal(h_0,std=0.0001)
        self.h_0 = nn.Parameter(h_0,requires_grad=True)
        self.linear = nn.Linear(600,300)
        self.drop = nn.Dropout(p1)
        self.gru = nn.GRU(input_size=300,hidden_size=300,num_layers=1,bias=True,batch_first=False)
        self.prob = nn.Linear(300,1)
    def forward(self,example):
        article_vector = None
        for sentence in example.nest_TREE:
            stack = list()
            for token in (sentence):
                if token != ')':
                    stack.append(self.embedding[self.stoi[token]].view(-1,300))
                else:
                    a1 = stack.pop()
                    a2 = stack.pop()
                    a = torch.cat((a1,a2),dim=1)
                    a = self.linear(a)
                    a = self.drop(a)
                    stack.append(nn.functional.tanh(a))
            sentence_vector = stack.pop()
            if article_vector is None:
                article_vector = sentence_vector.clone()
            else:
                article_vector = torch.cat((article_vector,sentence_vector),dim=0)
                
        seq,input_size = article_vector.shape
        output,h_n = self.gru(article_vector.view(seq,1,input_size),self.h_0)
        h_n = h_n.view(-1)
        out = self.prob(h_n)
        return nn.functional.sigmoid(out)
    
    
    
    
def fit(epoch,model,examples,group = 100):
    model.train()
    total_loss = 0
    indices = np.arange(len(examples))
    np.random.shuffle(indices)
    np.random.shuffle(indices)
    indices = indices[::-1]
    indices = np.array_split(indices,group)
    
    for index in indices:
        loss = torch.FloatTensor([0]).cuda()
        for e in index:
            example = examples[e]
            target = torch.FloatTensor([int(example.target)]).cuda()
            prob = model(example)
            loss += nn.functional.binary_cross_entropy(prob,target,size_average=False)
        optimizer.zero_grad()
        total_loss += loss.item()
        loss = (1/len(index))*loss
        loss.backward()
        optimizer.step()
    print(epoch+1,'th training end')
    return

    
def pred(epoch,model,examples):
    model.eval()
    pred = np.array([])
    for example in examples:
        prob = model(example)
        target = np.array([int(example.target)])
        prob = np.array([prob.item()])
        pred = np.append(pred,prob)
    a = pred[::2]
    b = pred[1::2]
    correct_rate = ((a>b).sum())/len(a)
    print(epoch+1,'th test accuracuy:',correct_rate)
    return
  


# Train+Test on earthquake dataset

In [176]:
#TEXT = data.Field(sequential=True,lower=True,use_vocab=True)
TREE = data.Field(sequential=False,use_vocab=False,preprocessing=fun,lower=True)
#NEST_TEXT = data.NestedField(TEXT,tokenize=split_n)
NEST_TREE = data.NestedField(TREE,tokenize=split_n,use_vocab=False)
TARGET = data.Field(sequential=False)
earthquake_train = data.TabularDataset(path='../finaldataset1/earthquake_train.csv',format='csv',fields=[('nest_TREE',NEST_TREE),
                                                                   ('target',TARGET)])
earthquake_test = data.TabularDataset(path='../finaldataset1/earthquake_test.csv',format='csv',fields=[('nest_TREE',NEST_TREE),
                                                                   ('target',TARGET)])

print('The number of pairs of articles we have for the training set:', len(earthquake_train.examples)/2)
print('The number of pairs of articles we have for the testing set:' ,len(earthquake_test.examples)/2)

treeRnnAndGru = TreeRnnAndGRU(p1=0.5)
treeRnnAndGru = treeRnnAndGru.cuda()
optimizer = torch.optim.Adam(treeRnnAndGru.parameters(),lr=0.005,weight_decay=0)
for epoch in range(20):
    fit(epoch,treeRnnAndGru,earthquake_train.examples)
    pred(epoch,treeRnnAndGru,earthquake_test.examples)

The number of pairs of articles we have for the training set: 1490.0
The number of pairs of articles we have for the testing set: 1720.0
1 th training end
1 th test accuracuy: 0.8453488372093023
2 th training end
2 th test accuracuy: 0.9877906976744186
3 th training end
3 th test accuracuy: 0.9953488372093023
4 th training end
4 th test accuracuy: 0.9936046511627907
5 th training end
5 th test accuracuy: 0.9843023255813953
6 th training end
6 th test accuracuy: 0.9965116279069768
7 th training end
7 th test accuracuy: 0.9941860465116279
8 th training end
8 th test accuracuy: 0.997093023255814
9 th training end
9 th test accuracuy: 0.9906976744186047
10 th training end
10 th test accuracuy: 0.9953488372093023
11 th training end
11 th test accuracuy: 0.9930232558139535
12 th training end
12 th test accuracuy: 0.9947674418604651
13 th training end
13 th test accuracuy: 0.9808139534883721
14 th training end
14 th test accuracuy: 0.9959302325581395
15 th training end
15 th test accuracuy: 0

# Train+Test on airplane dataset

In [179]:
TREE = data.Field(sequential=False,use_vocab=False,preprocessing=fun,lower=True)
#NEST_TEXT = data.NestedField(TEXT,tokenize=split_n)
NEST_TREE = data.NestedField(TREE,tokenize=split_n,use_vocab=False)
TARGET = data.Field(sequential=False)
airplane_train = data.TabularDataset(path='../finaldataset1/airplane_train.csv',format='csv',fields=[('nest_TREE',NEST_TREE),
                                                                   ('target',TARGET)])
airplane_test = data.TabularDataset(path='../finaldataset1/airplane_test.csv',format='csv',fields=[('nest_TREE',NEST_TREE),
                                                                   ('target',TARGET)])

print('The number of pairs of articles we have for the training set:', len(airplane_train.examples)/2)
print('The number of pairs of articles we have for the testing set:' ,len(airplane_test.examples)/2)

treeRnnAndGru = TreeRnnAndGRU(p1=0.5)
treeRnnAndGru = treeRnnAndGru.cuda()
optimizer = torch.optim.Adam(treeRnnAndGru.parameters(),lr=0.001,weight_decay=0)
for epoch in range(20):
    fit(epoch,treeRnnAndGru,airplane_train.examples,group=120)
    pred(epoch,treeRnnAndGru,airplane_test.examples)

The number of pairs of articles we have for the training set: 1960.0
The number of pairs of articles we have for the testing set: 1990.0
1 th training end
1 th test accuracuy: 0.8683417085427135
2 th training end
2 th test accuracuy: 0.8547738693467337
3 th training end
3 th test accuracuy: 0.8648241206030151
4 th training end
4 th test accuracuy: 0.8824120603015075
5 th training end
5 th test accuracuy: 0.878391959798995
6 th training end
6 th test accuracuy: 0.885929648241206
7 th training end
7 th test accuracuy: 0.8577889447236181
8 th training end
8 th test accuracuy: 0.8899497487437186
9 th training end
9 th test accuracuy: 0.8914572864321608
10 th training end
10 th test accuracuy: 0.8839195979899498
11 th training end
11 th test accuracuy: 0.893467336683417
12 th training end
12 th test accuracuy: 0.8693467336683417
13 th training end
13 th test accuracuy: 0.892964824120603
14 th training end
14 th test accuracuy: 0.8984924623115578
15 th training end
15 th test accuracuy: 0.88

Best accuracy for earthquake is <b>0.9965</b>

Best accuracy for airplane accident is <b>0.898</b>

Below is some other famous models performance on the same dataset


<img src="./result.png" height="800" width="1000">

In [10]:
path = '../finaldataset1/'
save_path = '../recursive data/'
for name in os.listdir(path):
    if re.findall('^[a-z0-9]',name):
        df = pd.read_csv(path+name ,header=None)
        df.columns = ['binary','target']
        df['binary'] = df['binary'].apply(lambda x: x.replace('(','').replace(')',''))
        df.to_csv(save_path+name,header=False,index=False)

In [5]:
class RnnAndGRU(nn.Module):
    def __init__(self,p1=0.2):
        super().__init__()
        stoi = open('../vocabulary/stoi.pickle','rb')
        self.stoi = pickle.load(stoi)
        embedding = torch.load('../vocabulary/embedding')
        self.embedding = embedding.cuda()
        h_0 = torch.zeros(1,1,300)
        h_0 = torch.normal(h_0,std=0.0001)
        self.h_0 = nn.Parameter(h_0,requires_grad=True)
        h_rnn = torch.zeros(1,300)
        h_rnn = torch.normal(h_rnn,std=0.0001)
        self.h_rnn = nn.Parameter(h_rnn,requires_grad=True)
        
        self.linear = nn.Linear(600,300)
        self.drop = nn.Dropout(p1)
        self.gru = nn.GRU(input_size=300,hidden_size=300,num_layers=1,bias=True,batch_first=False)
        self.prob = nn.Linear(300,1)
    def forward(self,example):
        article_vector = None
        for sentence in example.sentences:
            h = self.h_rnn
            for token in sentence:
                e_t = self.embedding[self.stoi[token]].view(-1,300)
                h = nn.functional.tanh(self.linear(torch.cat((e_t,h),dim=1)))
            if article_vector is None:
                article_vector = h.clone()
            else:
                article_vector = torch.cat((article_vector,h),dim=0)
        seq,input_size = article_vector.shape
        output,h_n = self.gru(article_vector.view(seq,1,input_size),self.h_0)
        h_n = h_n.view(-1)
        out = self.prob(h_n)
        return nn.functional.sigmoid(out)
    
def fit(epoch,model,examples,group = 100):
    model.train()
    total_loss = 0
    indices = np.arange(len(examples))
    np.random.shuffle(indices)
    np.random.shuffle(indices)
    indices = indices[::-1]
    indices = np.array_split(indices,group)
    
    for index in indices:
        loss = torch.FloatTensor([0]).cuda()
        #loss = torch.FloatTensor([0])
        for e in index:
            example = examples[e]
            target = torch.FloatTensor([int(example.target)]).cuda()
            #target = torch.FloatTensor([int(example.target)])
            prob = model(example)
            loss += nn.functional.binary_cross_entropy(prob,target,size_average=False)
        optimizer.zero_grad()
        total_loss += loss.item()
        loss = (1/len(index))*loss
        loss.backward()
        optimizer.step()
    print(epoch+1,'th training end')
    return

    
def pred(epoch,model,examples):
    model.eval()
    pred = np.array([])
    for example in examples:
        prob = model(example)
        target = np.array([int(example.target)])
        prob = np.array([prob.item()])
        pred = np.append(pred,prob)
    a = pred[::2]
    b = pred[1::2]
    correct_rate = ((a>b).sum())/len(a)
    print(epoch+1,'th test accuracuy:',correct_rate)
    return
  

In [8]:
sentence = data.Field(sequential=True,use_vocab=False,lower=True)
#NEST_TEXT = data.NestedField(TEXT,tokenize=split_n)
sentences = data.NestedField(sentence,tokenize=split_n,use_vocab=False)
TARGET = data.Field(sequential=False)
airplane_train = data.TabularDataset(path='../recursive data/airplane_train.csv',format='csv',fields=[('sentences',sentences),
                                                                   ('target',TARGET)])
airplane_test = data.TabularDataset(path='../recursive data/airplane_test.csv',format='csv',fields=[('sentences',sentences),
                                                                   ('target',TARGET)])

print('The number of pairs of articles we have for the training set:', len(airplane_train.examples)/2)
print('The number of pairs of articles we have for the testing set:' ,len(airplane_test.examples)/2)


rnnAndGRU = RnnAndGRU(p1=0.5)
rnnAndGRU = rnnAndGRU.cuda()
optimizer = torch.optim.Adam(rnnAndGRU.parameters(),lr=0.001,weight_decay=0)
for epoch in range(20):
    fit(epoch,rnnAndGRU,airplane_train.examples,group=120)
    pred(epoch,rnnAndGRU,airplane_test.examples)

The number of pairs of articles we have for the training set: 1960.0
The number of pairs of articles we have for the testing set: 1990.0
1 th training end
1 th test accuracuy: 0.8246231155778895
2 th training end
2 th test accuracuy: 0.749748743718593
3 th training end
3 th test accuracuy: 0.8175879396984924
4 th training end
4 th test accuracuy: 0.7914572864321608
5 th training end
5 th test accuracuy: 0.828643216080402
6 th training end
6 th test accuracuy: 0.8417085427135679
7 th training end
7 th test accuracuy: 0.8301507537688442
8 th training end
8 th test accuracuy: 0.835678391959799
9 th training end
9 th test accuracuy: 0.8366834170854272
10 th training end
10 th test accuracuy: 0.8361809045226131
11 th training end
11 th test accuracuy: 0.8371859296482412
12 th training end
12 th test accuracuy: 0.8386934673366834
13 th training end
13 th test accuracuy: 0.8396984924623115
14 th training end
14 th test accuracuy: 0.8391959798994975
15 th training end
15 th test accuracuy: 0.8

KeyboardInterrupt: 

In [11]:
sentence = data.Field(sequential=True,use_vocab=False,lower=True)
#NEST_TEXT = data.NestedField(TEXT,tokenize=split_n)
sentences = data.NestedField(sentence,tokenize=split_n,use_vocab=False)
TARGET = data.Field(sequential=False)

earthquake_train = data.TabularDataset(path='../recursive data/earthquake_train.csv',format='csv',fields=[('sentences',sentences),
                                                                   ('target',TARGET)])
earthquake_test = data.TabularDataset(path='../recursive data/earthquake_test.csv',format='csv',fields=[('sentences',sentences),
                                                                   ('target',TARGET)])

print('The number of pairs of articles we have for the training set:', len(earthquake_train.examples)/2)
print('The number of pairs of articles we have for the testing set:' ,len(earthquake_test.examples)/2)

rnnAndGRU = RnnAndGRU(p1=0.5)
rnnAndGRU = rnnAndGRU.cuda()
optimizer = torch.optim.Adam(rnnAndGRU.parameters(),lr=0.001,weight_decay=0)
for epoch in range(20):
    fit(epoch,rnnAndGRU,earthquake_train.examples,group=90)
    pred(epoch,rnnAndGRU,earthquake_test.examples)

The number of pairs of articles we have for the training set: 1490.0
The number of pairs of articles we have for the testing set: 1720.0
1 th training end
1 th test accuracuy: 0.7854651162790698
2 th training end
2 th test accuracuy: 0.8517441860465116
3 th training end
3 th test accuracuy: 0.9430232558139535
4 th training end
4 th test accuracuy: 0.7343023255813953
5 th training end
5 th test accuracuy: 0.7517441860465116
6 th training end
6 th test accuracuy: 0.7261627906976744
7 th training end
7 th test accuracuy: 0.7546511627906977
8 th training end
8 th test accuracuy: 0.7825581395348837
9 th training end
9 th test accuracuy: 0.7610465116279069
10 th training end
10 th test accuracuy: 0.7325581395348837
11 th training end


KeyboardInterrupt: 

In [18]:
sentence = data.Field(sequential=True,use_vocab=False,lower=True)
#NEST_TEXT = data.NestedField(TEXT,tokenize=split_n)
sentences = data.NestedField(sentence,tokenize=split_n,use_vocab=False)
TARGET = data.Field(sequential=False)

earthquake_train = data.TabularDataset(path='../recursive data/earthquake_train.csv',format='csv',fields=[('sentences',sentences),
                                                                   ('target',TARGET)])
earthquake_test = data.TabularDataset(path='../recursive data/earthquake_test.csv',format='csv',fields=[('sentences',sentences),
                                                                   ('target',TARGET)])

print('The number of pairs of articles we have for the training set:', len(earthquake_train.examples)/2)
print('The number of pairs of articles we have for the testing set:' ,len(earthquake_test.examples)/2)

rnnAndGRU = RnnAndGRU(p1=0.5)
rnnAndGRU = rnnAndGRU.cuda()
optimizer = torch.optim.Adam(rnnAndGRU.parameters(),lr=0.001,weight_decay=0)
for epoch in range(20):
    fit(epoch,rnnAndGRU,earthquake_train.examples,group=90)
    pred(epoch,rnnAndGRU,earthquake_test.examples)

The number of pairs of articles we have for the training set: 1490.0
The number of pairs of articles we have for the testing set: 1720.0
1 th training end
1 th test accuracuy: 0.7906976744186046
2 th training end
2 th test accuracuy: 0.9436046511627907
3 th training end
3 th test accuracuy: 0.8941860465116279
4 th training end
4 th test accuracuy: 0.8372093023255814
5 th training end
5 th test accuracuy: 0.852906976744186
6 th training end
6 th test accuracuy: 0.7715116279069767
7 th training end
7 th test accuracuy: 0.7604651162790698
8 th training end
8 th test accuracuy: 0.7761627906976745
9 th training end
9 th test accuracuy: 0.7854651162790698


KeyboardInterrupt: 